In [ ]:
#Import Libraries
import pandas as pd
import openpyxl
from random import randint, shuffle

In [ ]:
#Name of the Excel file and sheets to be used
file_name = r'SecretSantaDataSet.xlsx'
people_sheet = r'Pessoas'
incompatibles_sheet = r'Incompatibilidades'

In [ ]:
#Import participants from excel and put them on a list
people = pd.read_excel(file_name, engine='openpyxl', sheet_name=people_sheet)
people = people.iloc[:, 0]

In [ ]:
#Import incompatibilities between participants and put them on a list of sets
incompatibilities = pd.read_excel(file_name, engine='openpyxl', sheet_name=incompatibles_sheet)
incompats = []
for index, row in incompatibilities.iterrows():
    a_set = {row[0], row[1]}
    incompats.append(a_set)

In [ ]:
#FUNCTION FIND_INCOMPATIBILITIES
#Returns a set of people that a "name_set" cannot match with, based on the incompats list
#Returns  subset of the incompats list for a person
def find_incompatibilities(name_set, exclusion_list): 
    incompatibles = set()
    for i in range(len(exclusion_list)):
        if name_set.issubset(exclusion_list[i]):
            incompatibles = incompatibles.union(exclusion_list[i])
    return incompatibles

In [ ]:
#FUNCTION FIND_POSSIBILITIES
#Returns the list of possible matches for a person, excluding themselves
#Returns the list of the difference between all participants and the incompatibilities for a person
def find_possibilities(name_set, receivers_set, exclusion_list): 
    possibilities_set = set()
    possibilities_set = receivers_set.difference(name_set)
    possibilities_set = possibilities_set.difference(find_incompatibilities(name_set, exclusion_list))
    return list(possibilities_set)

In [ ]:
#Program
#Initial validations
end_program = False
max_tries = 1000
tries = 0
if len(people) <2:
    print("Secret Santa needs at least 2 participants.")
    end_program = True

elif people.isnull().any:
    print("The participants list has null values.")
    end_Program = True

elif len(people) != len(set(people)):
    duplicates = [x for x in people if people.count(x)>1]
    print(f'There can\'t be participants with exactly the same name.\nThese are repeated: {set(duplicates)}')
    end_Program = True

#SortingHat
else:
    people = list(people)
    while tries < max_tries:
        shuffle(people)
        givers = people.copy()
        receivers = people.copy()
        matches = []
        for p in people :
            #Gets a list of all possible receivers for a participant p
            all_possible_rec = find_possibilities({givers[0]}, set(receivers), incompats)
            if len(all_possible_rec) == 0 :
                break
            #Selects a random match from all possible receivers
            match = all_possible_rec[randint(0, len(all_possible_rec)-1)]
            matches.append([givers[0], match]) 
            givers.pop(0)
            receivers.pop(receivers.index(match))
        tries += 1
        if len(matches) == len(people):
            break

In [ ]:
#Converts results list into data frame and prints results
df_matches = []
if not end_program and tries != 0:    
        print(f'{tries} tries; {len(matches)} out of {len(people)} matches')

        if tries == max_tries and len(matches) != len(people):
            print(f'Maybe try again next year with more participants and less restrictions.')
            end_Program = True

        else:
            df_matches = pd.DataFrame(matches, columns=['Gift Giver', 'Gift Receiver'])
            print(df_matches)

In [ ]:
#Writes results to a new sheet on Excel File
wb = openpyxl.load_workbook(file_name)
if 'Resultados' in wb.sheetnames:
    wb.remove(wb['Resultados'])

if not end_program and tries != 0 and len(df_matches) != 0:

        ws_new = wb.create_sheet('Resultados')
        ws_new.append(list(df_matches.columns))
        for i in range(len(df_matches)):
            ws_new.append(list(df_matches.loc[i, :]))

        ws_new.column_dimensions['A'].width = 55
        ws_new.column_dimensions['B'].width = 55

wb.save(file_name)
wb.close()
